In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 27.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import TFAutoModel, AutoTokenizer

train_df = pd.read_csv('drive/MyDrive/liar_dataset/train.tsv',sep='\t', header=None, names=['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'])
test_df = pd.read_csv('drive/MyDrive/liar_dataset/test.tsv',sep='\t', header=None, names=['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'])
val_df = pd.read_csv('drive/MyDrive/liar_dataset/valid.tsv',sep='\t', header=None, names=['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'])


df = pd.concat([train_df, test_df], axis=0)

In [ ]:
df = pd.concat([train_df, test_df], axis=0)

tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased')
X = df['statement'].values
X = tokenizer(X.tolist(), padding=True, truncation=True, max_length=128, return_tensors='tf')
y = pd.get_dummies(df['label']).values

X_train = [X['input_ids'].numpy()[0:train_df.shape[0]], X['attention_mask'].numpy()[0:train_df.shape[0]]]
y_train = y[:train_df.shape[0]]
X_test = [X['input_ids'].numpy()[train_df.shape[0]:], X['attention_mask'].numpy()[train_df.shape[0]:]]
y_test = y[train_df.shape[0]:]

bert_model = TFAutoModel.from_pretrained('bert-large-uncased')

input_ids = tf.keras.layers.Input(shape=(128,), dtype='int32')
input_masks = tf.keras.layers.Input(shape=(128,), dtype='int32')
bert_output = bert_model({'input_ids': input_ids, 'attention_mask': input_masks})[0]
output_layer = tf.keras.layers.Dense(6, activation='softmax')(bert_output[:, 0, :])
model = tf.keras.models.Model(inputs=[input_ids, input_masks], outputs=output_layer)
for layer in model.layers[:4]:
    layer.trainable = False


Some layers from the model checkpoint at bert-large-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-large-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
import keras.backend as K
def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 20*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', f1_metric])
X_train = np.array(X_train)
input = (X_train[0], X_train[1])
y_train = np.array(y_train)
input1 = (X_test[0],X_test[1])
history = model.fit(input, y_train, epochs=3, batch_size=32)

score = model.evaluate(X_test, y_test, batch_size=32)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/3
320/320 [==============================] - 298s 879ms/step - loss: 1.6758 - accuracy: 0.2969 - f1_metric: 0.5590
Epoch 2/3
320/320 [==============================] - 281s 879ms/step - loss: 1.6671 - accuracy: 0.2956 - f1_metric: 0.4643
Epoch 3/3
40/40 [==============================] - 42s 866ms/step - loss: 1.7495 - accuracy: 0.2415 - f1_metric: 0.4729
Test loss: 1.7494585514068604
Test accuracy: 0.2415153980255127


In [ ]:
print(score)

[1.699894666671753, 0.26598262786865234, 0.03094685636460781]


In [ ]:
y_pred = model.predict(X_test)